In [1]:
import xarray as xr
import s3fs
import zarr
from dask.distributed import Client

client = Client()
client.get_versions(check=True)
client

Client Scheduler: tcp://singed-sabertooth-dask-scheduler:8786 Dashboard: http://singed-sabertooth-dask-scheduler:8787/status,Cluster Workers: 3 Cores: 192 Memory: 811.41 GB


### Test 5

* 4 Variables: time, lat, lon, analysed_sst
* 1 dataset
* 2 m4.16xlarge ec2 instances

In [2]:
%%time
sst_netcdf = 'https://podaac-opendap.jpl.nasa.gov:443/opendap/allData/ghrsst/data/GDS2/L4/GLOB/JPL/MUR/v4.1/2002/152/20020601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc?time[0:1:0],lat[0:1:17998],lon[0:1:35999],analysed_sst[0:1:0][0:1:17998][0:1:35999]'
ds = xr.open_dataset(sst_netcdf, chunks={'time': 366, 'lat': 250, 'lon': 500})
compressor = zarr.Blosc(cname='zstd', clevel=5, shuffle=zarr.Blosc.AUTOSHUFFLE)
encoding = {v: {'compressor': compressor} for v in ds.data_vars}

f_zarr = 'ds-data-projects/mur_sst'
fs = s3fs.S3FileSystem(anon=False)
d = s3fs.S3Map(f_zarr, s3=fs)
ds.to_zarr(store=d, mode='w', encoding=encoding)

CPU times: user 1min 49s, sys: 2.7 s, total: 1min 52s
Wall time: 20min 51s
